In [273]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.simplefilter('ignore')

# 카카오톡스토어의 상품 리스트를 다운로드
df = pd.read_excel('/Users/seanyu_mac/Downloads/product_20250226101648.xlsx')
# 현재 판매중인 건들만 필터링
df = df[df['판매상태'] == '판매중']
# 약사별 판매링크 아닌 공동구매 건들 필터링
df.drop(df[df['상품명'].str.contains('공동구매')].index, inplace=True)
# 약사별 판매링크 아닌 것들은 필터링
df.drop(df[df['상품명'].str.contains(']') == False].index, inplace=True)
# 중복된 건 필터링 : 풀스펙멀티비타민 친약, 친한약사
df.drop(df[df['상품번호'] == 373500859].index, inplace=True)
df.reset_index(drop=True, inplace=True)

# pb_product 테이블 다운로드 및 naver 관련 상품들은 제거
pb_df = pd.read_csv('/Users/seanyu_mac/Downloads/public_pb_product_export_2025-02-26_102450.csv')
pb_df.drop(pb_df[pb_df['product_id'].str.contains('naver')].index, inplace=True)
pb_df.drop(401, inplace=True)

# 친한스토어 상품 등록 시, 필요한 정보
# product_id, 컬렉션, 판매기간, 상품명, 기본옵션, 판매옵션, 배송비 정책, 배송택배사, 출고지, 반품/교환 택배사, 반품/교환지, 썸네일, 상세 이미지, 고시 이미지, 상품상태, 원산지, 영수증 발행여부, 제품명, 식품의 유형, 생산지 및 소재지, 제조연월일 소비기한 또는 품질 유지기한, 포장단위별 내용량의 용량(중량) 수량, 원재료명 및 함량, 영양성분, 유전자변형식품에 해당, 소비자안전을 위한 주의사항, 수입식품에 해당

In [275]:
product_link = {}
yaksa_names = []
product_names = []

for i in df['상품명'].unique():
    product_name = i.split(']')[1].strip()
    product_names.append(product_name)
    yaksa_name1 = i.split(']')[0].strip()
    yaksa_name2 = yaksa_name1.split('/')[1].strip()
    yaksa_names.append(yaksa_name2)

idx = sorted(list(set(product_names)))

In [277]:
ids = []

for i in idx:
    i = i.split()
    i = i[0] + ' ' + i[1]
    if '(' in i:
        i = i.split('(')[0]
    elif '1박스' in i:
        i = i.split('1박스')[0]
    ids.append(i)

for i in range(len(ids)):
    if '썬비타민' in ids[i]:
        ids[i] = 'YDY 썬 비타민'
    elif '코랄칼마디' in ids[i]:
        ids[i] = '코랄칼마디 K2'
    elif '포스트바이오틱스' in ids[i]:
        ids[i] = 'YDY 포스트바이오틱스 GX'
    elif '풀스펙멀티비타민' in ids[i]:
        ids[i] = 'YDY 풀스펙 멀티비타민'
    elif '퓨어리포좀글루타치온' in ids[i]:
        ids[i] = 'YDY 퓨어 리포좀 글루타치온'
    elif '퓨어리포좀비타민' in ids[i]:
        ids[i] = 'YDY 퓨어 리포좀 비타민C'
    elif '액티브 헴철' in ids[i]:
        ids[i] = 'YDY 액티브헴철'
    elif '액티브비큐텐' in ids[i]:
        ids[i] = 'YDY 액티브비큐텐'
    elif '오큐클리어' in ids[i]:
        ids[i] = 'YDY 오큐클리어'
    elif '폴라초임계오메가3' in ids[i]:
        ids[i] = 'YDY 폴라초임계오메가3'
    elif '마그듀오' in ids[i]:
        ids[i] = 'YDY 마그듀오'

In [279]:
pb_name_idx = []

for i in ids:
    con = pb_df['product_name'].str.contains(i)
    pb_name_idx.append(pb_df.loc[con, 'product_name'].values[0])

In [293]:
keys = range(len(df.columns))  # No need for (0, len(...)), range(len(...)) works
values = df.columns.tolist()  # Get list of column names
col_dict = dict(zip(keys, values))  # Use a different variable name instead of "dict"

print(col_dict)

TypeError: 'dict' object is not callable

In [283]:
result_df = pd.DataFrame()
txt = 'product_id, 컬렉션, 판매기간, 상품명, 기본옵션, 판매옵션, 약사님, 배송비 정책, 배송택배사, 출고지, 반품/교환 택배사, 반품/교환지, 썸네일, 상세 이미지, 고시 이미지, 상품상태, 원산지, 영수증 발행여부, 제품명, 식품의 유형, 생산지 및 소재지, 제조연월일 소비기한 또는 품질 유지기한, 포장단위별 내용량의 용량(중량) 수량, 원재료명 및 함량, 영양성분, 유전자변형식품에 해당, 소비자안전을 위한 주의사항, 수입식품에 해당'
txt = txt.split(',')
txts = []

for i in txt:
    txts.append(i.strip())

for i in txts:
    result_df[i] = 0

In [8]:
data = []
for i in range(630):
    data.append('공동구매')
result_df['컬렉션'] = data 

data = []
for i in range(630):
    data.append('2025-02-28 ~ 2099-12-31')
result_df['판매기간'] = data

data = []
for i in range(630):
    data.append('2025-02-28 ~ 2099-12-31')
result_df['판매기간'] = data

result_df['상품명'] = df['상품명']
result_df['배송비 정책'] = '무료배송'
result_df['배송택배사'] = '파마브로스'
result_df['배송택배사'] = '파마브로스'
result_df['출고지'] = '파마브로스'
result_df['반품/교환 택배사'] = '파마브로스'
result_df['반품/교환지'] = '파마브로스'
result_df['상품상태'] = '새상품'
result_df['원산지'] = '국내산'
result_df['영수증 발행여부'] = '신용카드/현금영수증 발급'

In [9]:
result_df['수입식품에 해당'] = '해당없음'

for i in ['제품명',
 '식품의 유형',
 '생산지 및 소재지',
 '제조연월일 소비기한 또는 품질 유지기한',
 '포장단위별 내용량의 용량(중량) 수량',
 '원재료명 및 함량',
 '영양성분',
 '유전자변형식품에 해당',
 '소비자안전을 위한 주의사항'
 ]:
    result_df[i] = '상품상세설명 참조'

In [10]:
yaksas = []

for i in result_df['상품명']:
    i = i.split(']')[0]
    i = i.split('/')[1]
    yaksas.append(i.strip())

result_df['약사님'] = yaksas

In [11]:
result_df.loc[result_df['약사님'] == '배혜정약사', '약사님'] = '우리약사'
result_df.loc[result_df['약사님'] == '김약사tv', '약사님'] = '김미성약사'
result_df.loc[result_df['약사님'] == '김약사TV', '약사님'] = '김미성약사'
result_df.loc[result_df['약사님'] == '친한', '약사님'] = '친한약사'

In [12]:
result_df.loc[:, '판매옵션'] = '없음'

/var/folders/wl/txjbcgld2sngchsc7txxqppm0000gp/T/ipykernel_83562/1236317530.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '없음' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  result_df.loc[:, '판매옵션'] = '없음'


In [13]:
result_df

,product_id,컬렉션,판매기간,상품명,기본옵션,판매옵션,약사님,배송비 정책,배송택배사,출고지,...,제품명,식품의 유형,생산지 및 소재지,제조연월일 소비기한 또는 품질 유지기한,포장단위별 내용량의 용량(중량) 수량,원재료명 및 함량,영양성분,유전자변형식품에 해당,소비자안전을 위한 주의사항,수입식품에 해당
0,NaN,공동구매,2025-02-28 ~ 2099-12-31,[스페셜할인코드/서울약사] 프로바이오 에스엘비(SLB) 1박스,NaN,없음,서울약사,무료배송,파마브로스,파마브로스,...,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,해당없음
1,NaN,공동구매,2025-02-28 ~ 2099-12-31,[스페셜할인코드/서울약사] 퓨어리포좀비타민C 1박스,NaN,없음,서울약사,무료배송,파마브로스,파마브로스,...,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,해당없음
2,NaN,공동구매,2025-02-28 ~ 2099-12-31,[스페셜할인코드/서울약사] 퓨어리포좀글루타치온 1박스,NaN,없음,서울약사,무료배송,파마브로스,파마브로스,...,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,해당없음
3,NaN,공동구매,2025-02-28 ~ 2099-12-31,[스페셜할인코드/서울약사] 풀스펙멀티비타민 1박스,NaN,없음,서울약사,무료배송,파마브로스,파마브로스,...,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,해당없음
4,NaN,공동구매,2025-02-28 ~ 2099-12-31,[스페셜할인코드/서울약사] 폴라초임계오메가3 1박스,NaN,없음,서울약사,무료배송,파마브로스,파마브로스,...,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,해당없음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,NaN,공동구매,2025-02-28 ~ 2099-12-31,[스페셜할인코드/약사임당] YDY 오큐클리어 1박스(1개월분),NaN,없음,약사임당,무료배송,파마브로스,파마브로스,...,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,해당없음
626,NaN,공동구매,2025-02-28 ~ 2099-12-31,[스페셜할인코드/위키약사] YDY 오큐클리어 1박스(1개월분),NaN,없음,위키약사,무료배송,파마브로스,파마브로스,...,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,해당없음
627,NaN,공동구매,2025-02-28 ~ 2099-12-31,[스페셜할인코드/릴리약사] YDY 오큐클리어 1박스(1개월분),NaN,없음,릴리약사,무료배송,파마브로스,파마브로스,...,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,해당없음
628,NaN,공동구매,2025-02-28 ~ 2099-12-31,[스페셜할인코드/유이약사] YDY 오큐클리어 1박스(1개월분),NaN,없음,유이약사,무료배송,파마브로스,파마브로스,...,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,상품상세설명 참조,해당없음


In [14]:
pb_nums = []

for i in ids:
    con = pb_df['product_name'].str.contains(i)
    pb_nums.append(pb_df.loc[con, 'id'].values[0])

In [15]:
new_dict =  {}
for i in range(len(ids)):
    new_dict[ids[i]] = pb_nums[i]
new_dict

{'YDY 내몸에': 27487,
 'YDY 마그듀오': 25118,
 'YDY 썬 비타민': 35514,
 'YDY 액티브비큐텐': 24271,
 'YDY 액티브헴철': 35450,
 'YDY 오큐클리어': 25123,
 '코랄칼마디 K2': 35713,
 'YDY 파이토': 26759,
 'YDY 포스트바이오틱스 GX': 35462,
 'YDY 폴라초임계오메가3': 25717,
 'YDY 풀스펙 멀티비타민': 34708,
 'YDY 퓨어': 35629,
 'YDY 퓨어 리포좀 글루타치온': 34910,
 'YDY 퓨어 리포좀 비타민C': 26214,
 'YDY 프로바이오': 26671,
 '내몸에 효소가득': 27487,
 '라이버 맥주효모': 35506,
 '이뮨 빌베리': 35585,
 '파이토 레드큐민': 26759,
 '파이토알엑스 라이버': 35506,
 '파이토알엑스 이뮨': 35585,
 '프로바이오 에스엘비': 26671}

In [34]:
result_df.loc[result_df['상품명'].str.contains('마그듀오'), 'product_id'] = 25118
result_df.loc[result_df['상품명'].str.contains('내몸에'), 'product_id'] = 27487
result_df.loc[result_df['상품명'].str.contains('썬'), 'product_id'] = 35514
result_df.loc[result_df['상품명'].str.contains('큐텐'), 'product_id'] = 24271
result_df.loc[result_df['상품명'].str.contains('헴철'), 'product_id'] = 35450
result_df.loc[result_df['상품명'].str.contains('오큐클리어'), 'product_id'] = 25123
result_df.loc[result_df['상품명'].str.contains('코랄칼마디'), 'product_id'] = 35713
result_df.loc[result_df['상품명'].str.contains('파이토 레드큐민'), 'product_id'] = 26759
result_df.loc[result_df['상품명'].str.contains('포스트바이오틱스'), 'product_id'] = 35462
result_df.loc[result_df['상품명'].str.contains('폴라'), 'product_id'] = 25717
result_df.loc[result_df['상품명'].str.contains('퓨어 레드큐민'), 'product_id'] = 35629
result_df.loc[result_df['상품명'].str.contains('글루타치온'), 'product_id'] = 34910
result_df.loc[result_df['상품명'].str.contains('비타민C'), 'product_id'] = 26214
result_df.loc[result_df['상품명'].str.contains('프로바이오'), 'product_id'] = 26671
result_df.loc[result_df['상품명'].str.contains('라이버'), 'product_id'] = 35506
result_df.loc[result_df['상품명'].str.contains('이뮨'), 'product_id'] = 35585
result_df.loc[result_df['상품명'].str.contains('멀티비타민'), 'product_id'] = 34708

In [68]:
# 정상가, 판매가, 재고수량, 업체품목코드, 박스수량
products = []

for i in result_df['상품명'].unique().tolist():
    data = i.split(']')[1].strip()
    products.append(data)

result_df['products'] = products

In [98]:
result_df['기본옵션'] = '있음'
result_df[['정상가', '판매가', '재고수량', '업체품목코드', '박스수량']] = 0

result_df.loc[result_df['상품명'].str.contains('마그듀오'), ['정상가', '판매가', '재고수량', '업체품목코드', '박스수량']] = [25000, 23750, 99999, 201069000003, 1]
result_df.loc[result_df['상품명'].str.contains('내몸에'), ['정상가', '판매가', '재고수량', '업체품목코드', '박스수량']] = [35000, 28500, 99999, 201067000007, 1]
result_df.loc[result_df['상품명'].str.contains('썬'), ['정상가', '판매가', '재고수량', '업체품목코드', '박스수량']] = [25000, 21370, 99999, 201067000008, 1]
result_df.loc[result_df['상품명'].str.contains('큐텐'), ['정상가', '판매가', '재고수량', '업체품목코드', '박스수량']] = [35000, 33250, 99999, 201069000004, 1]
result_df.loc[result_df['상품명'].str.contains('헴철'), ['정상가', '판매가', '재고수량', '업체품목코드', '박스수량']] = [40000, 34200, 99999, 201068000041, 1]
result_df.loc[result_df['상품명'].str.contains('오큐클리어'), ['정상가', '판매가', '재고수량', '업체품목코드', '박스수량']] = [45000, 42750, 99999, 201067000001, 1]
result_df.loc[result_df['상품명'].str.contains('코랄칼마디'), ['정상가', '판매가', '재고수량', '업체품목코드', '박스수량']] = [33000, 29450, 99999, 201807000036, 1]
result_df.loc[result_df['상품명'].str.contains('파이토 레드큐민'), ['정상가', '판매가', '재고수량', '업체품목코드', '박스수량']] = [55000, 48450, 99999, 20106700000, 1]
result_df.loc[result_df['상품명'].str.contains('포스트바이오틱스'), ['정상가', '판매가', '재고수량', '업체품목코드', '박스수량']] = [40000, 34200, 99999, 201807000011, 1]
result_df.loc[result_df['상품명'].str.contains('폴라'), ['정상가', '판매가', '재고수량', '업체품목코드', '박스수량']] = [28000, 26600, 99999, 201068000001, 1]
result_df.loc[result_df['상품명'].str.contains('퓨어 레드큐민'), ['정상가', '판매가', '재고수량', '업체품목코드', '박스수량']] = [44000, 41800, 99999, 201807000026, 1]
result_df.loc[result_df['상품명'].str.contains('글루타치온'), ['정상가', '판매가', '재고수량', '업체품목코드', '박스수량']] = [59000, 56050, 99999, 201067000012, 1]
result_df.loc[result_df['상품명'].str.contains('비타민C'), ['정상가', '판매가', '재고수량', '업체품목코드', '박스수량']] = [29000, 27550, 99999, 201067000002, 1]
result_df.loc[result_df['상품명'].str.contains('프로바이오'), ['정상가', '판매가', '재고수량', '업체품목코드', '박스수량']] = [55000, 47020, 99999, 201067000004, 1]
result_df.loc[result_df['상품명'].str.contains('라이버'), ['정상가', '판매가', '재고수량', '업체품목코드', '박스수량']] = [28000, 21700, 99999, 201807000016, 1]
result_df.loc[result_df['상품명'].str.contains('이뮨'), ['정상가', '판매가', '재고수량', '업체품목코드', '박스수량']] = [33900, 32200, 99999, 201807000020, 1]
result_df.loc[result_df['상품명'].str.contains('멀티비타민'), ['정상가', '판매가', '재고수량', '업체품목코드', '박스수량']] = [25000, 21370, 99999, 201067000013, 1]

In [108]:
magduo_thumb1 = 'https://img.friendly-pharmacist.com/plto_store_kakao/1_0.png'
bq10_thumb1 = 'https://img.friendly-pharmacist.com/plto_store_kakao/2_0.png'
ocuclear_thumb1 = 'https://img.friendly-pharmacist.com/YDY/OQclear_lutein/oqclear-thum_01.jpg'
omega3_thumb1 = 'https://img.friendly-pharmacist.com/plto_store_kakao/4_0.png'
vitaminc_thumb1 = 'https://img.friendly-pharmacist.com/plto_store_kakao/5_0.png'
coral_thumb1 = 'https://img.friendly-pharmacist.com/YDY/Coral_kal_K2/thum_coral_k2.jpg'
slb_thumb1 = 'https://img.friendly-pharmacist.com/plto_store_kakao/7_0.png'
red_cumin_thumb1 = 'https://img.friendly-pharmacist.com/plto_store_kakao/8_0.png'
hyoso_thumb1 = 'https://img.friendly-pharmacist.com/plto_store_kakao/10_0.png'
fullspec_thumb1 = 'https://img.friendly-pharmacist.com/plto_store_kakao/12_0.png'
glu_thumb1 = 'https://img.friendly-pharmacist.com/plto_store_kakao/13_0.png'
hemchul_thumb1 = 'https://img.friendly-pharmacist.com/plto_store_kakao/14_0.png'
postgx_thumb1 = 'https://img.friendly-pharmacist.com/plto_store_kakao/15_0.png'
rx_macju_thumb1 = 'https://img.friendly-pharmacist.com/plto_store_kakao/16_0.png'
rx_billberry_thumb1 = 'https://img.friendly-pharmacist.com/plto_store_kakao/17_0.png'
sun_thumb1 = 'https://img.friendly-pharmacist.com/YDY/renewal_sunvitamin_D/sun_vitamin_thumbnail.jpg'
pure_cumin_thumb1 = 'https://img.friendly-pharmacist.com/Thumbnail/red_cumin_thumbnail.jpg'

result_df.loc[result_df['상품명'].str.contains('마그듀오'), '썸네일1'] = magduo_thumb1
result_df.loc[result_df['상품명'].str.contains('내몸에'), '썸네일1'] = hyoso_thumb1
result_df.loc[result_df['상품명'].str.contains('썬'), '썸네일1'] = sun_thumb1
result_df.loc[result_df['상품명'].str.contains('큐텐'), '썸네일1'] = bq10_thumb1
result_df.loc[result_df['상품명'].str.contains('헴철'), '썸네일1'] = hemchul_thumb1
result_df.loc[result_df['상품명'].str.contains('오큐클리어'), '썸네일1'] = ocuclear_thumb1
result_df.loc[result_df['상품명'].str.contains('코랄칼마디'), '썸네일1'] = coral_thumb1
result_df.loc[result_df['상품명'].str.contains('파이토 레드큐민'), '썸네일1'] = red_cumin_thumb1
result_df.loc[result_df['상품명'].str.contains('포스트바이오틱스'), '썸네일1'] = postgx_thumb1
result_df.loc[result_df['상품명'].str.contains('폴라'), '썸네일1'] = omega3_thumb1
result_df.loc[result_df['상품명'].str.contains('퓨어 레드큐민'), '썸네일1'] = pure_cumin_thumb1
result_df.loc[result_df['상품명'].str.contains('글루타치온'), '썸네일1'] = glu_thumb1
result_df.loc[result_df['상품명'].str.contains('비타민C'), '썸네일1'] = vitaminc_thumb1
result_df.loc[result_df['상품명'].str.contains('프로바이오'), '썸네일1'] = slb_thumb1
result_df.loc[result_df['상품명'].str.contains('라이버'), '썸네일1'] = rx_macju_thumb1
result_df.loc[result_df['상품명'].str.contains('이뮨'), '썸네일1'] = rx_billberry_thumb1
result_df.loc[result_df['상품명'].str.contains('멀티비타민'), '썸네일1'] = fullspec_thumb1

In [132]:
magduo_details = ['https://img.friendly-pharmacist.com/YDY/magduo/magduo_01.jpg',
                  'https://img.friendly-pharmacist.com/YDY/magduo/magduo_02.jpg',
                  'https://img.friendly-pharmacist.com/YDY/magduo/magduo_03.jpg',
                  'https://img.friendly-pharmacist.com/YDY/magduo/magduo_04.jpg',
                  'https://img.friendly-pharmacist.com/YDY/magduo/magduo_05.jpg',
                  'https://img.friendly-pharmacist.com/YDY/magduo/magduo_06.jpg','없음','없음','없음'
                 ]
bq10_details = ['https://img.friendly-pharmacist.com/YDY/Active_BQ10_DFE/Active_BQ10_DFE_01.jpg', 
                'https://img.friendly-pharmacist.com/YDY/Active_BQ10_DFE/Active_BQ10_DFE_02.jpg',
                'https://img.friendly-pharmacist.com/YDY/Active_BQ10_DFE/Active_BQ10_DFE_03.jpg',
                'https://img.friendly-pharmacist.com/YDY/Active_BQ10_DFE/Active_BQ10_DFE_04.jpg',
                'https://img.friendly-pharmacist.com/YDY/Active_BQ10_DFE/Active_BQ10_DFE_05.jpg','없음','없음','없음','없음'
               ]
ocuclear_details = ['https://img.friendly-pharmacist.com/YDY/OQclear_lutein/oqclear_detail_01.jpg',
                    'https://img.friendly-pharmacist.com/YDY/OQclear_lutein/oqclear_detail_02.jpg',
                    'https://img.friendly-pharmacist.com/YDY/OQclear_lutein/oqclear_detail_03.jpg',
                    'https://img.friendly-pharmacist.com/YDY/OQclear_lutein/oqclear_detail_04.jpg',
                    'https://img.friendly-pharmacist.com/YDY/OQclear_lutein/oqclear_detail_05.jpg','없음','없음','없음','없음'
                   ]
omega3_details = ['https://img.friendly-pharmacist.com/YDY/polaris_omega3/polar_omega3_01.jpg',
                  'https://img.friendly-pharmacist.com/YDY/polaris_omega3/polar_omega3_02.jpg',
                  'https://img.friendly-pharmacist.com/YDY/polaris_omega3/polar_omega3_03.jpg',
                  'https://img.friendly-pharmacist.com/YDY/polaris_omega3/polar_omega3_04.jpg',
                  'https://img.friendly-pharmacist.com/YDY/polaris_omega3/polar_omega3_05.jpg','없음','없음','없음','없음'
                 ]
vitaminc_details = ['https://img.friendly-pharmacist.com/YDY/vitaminc_new_detail/pure_vitaminc_01.jpg',
                    'https://img.friendly-pharmacist.com/YDY/vitaminc_new_detail/pure_vitaminc_02.jpg',
                    'https://img.friendly-pharmacist.com/YDY/vitaminc_new_detail/pure_vitaminc_03.jpg',
                    'https://img.friendly-pharmacist.com/YDY/vitaminc_new_detail/pure_vitaminc_04.jpg','없음','없음','없음','없음','없음'
                   ]
coral_details = ['https://img.friendly-pharmacist.com/YDY/Coral_kal_K2/coral_cal_k2_01.jpg',
                 'https://img.friendly-pharmacist.com/YDY/Coral_kal_K2/coral_cal_k2_02.jpg',
                 'https://img.friendly-pharmacist.com/YDY/Coral_kal_K2/coral_cal_k2_03.jpg',
                 'https://img.friendly-pharmacist.com/YDY/Coral_kal_K2/coral_cal_k2_04.jpg','없음','없음','없음','없음','없음'
                ]
slb_details = ['https://img.friendly-pharmacist.com/YDY/probio_slb/probio_slb_01.jpg',
               'https://img.friendly-pharmacist.com/YDY/probio_slb/probio_slb_02.jpg',
               'https://img.friendly-pharmacist.com/YDY/probio_slb/probio_slb_03.jpg',
               'https://img.friendly-pharmacist.com/YDY/probio_slb/probio_slb_04.jpg',
               'https://img.friendly-pharmacist.com/YDY/probio_slb/probio_slb_05.jpg','없음','없음','없음','없음'
              ]
red_cumin_details = ['https://img.friendly-pharmacist.com/YDY/red_cumin/red_cumin_01.jpg',
                     'https://img.friendly-pharmacist.com/YDY/red_cumin/red_cumin_02.jpg',
                     'https://img.friendly-pharmacist.com/YDY/red_cumin/red_cumin_03.jpg',
                     'https://img.friendly-pharmacist.com/YDY/red_cumin/red_cumin_04.jpg',
                     'https://img.friendly-pharmacist.com/YDY/red_cumin/red_cumin_05.jpg','없음','없음','없음','없음'
                    ]
hyoso_details = ['https://img.friendly-pharmacist.com/YDY/enzyme_full/enzyme_full_01.jpg',
                 'https://img.friendly-pharmacist.com/YDY/enzyme_full/enzyme_full_02.jpg',
                 'https://img.friendly-pharmacist.com/YDY/enzyme_full/enzyme_full_03.jpg',
                 'https://img.friendly-pharmacist.com/YDY/enzyme_full/enzyme_full_04.jpg',
                 'https://img.friendly-pharmacist.com/YDY/enzyme_full/enzyme_full_05.jpg',
                 'https://img.friendly-pharmacist.com/YDY/enzyme_full/enzyme_full_06.jpg','없음','없음','없음'
                ]
fullspec_details = ['https://img.friendly-pharmacist.com/YDY/multi-vitamin_01.jpg',
                    'https://img.friendly-pharmacist.com/YDY/multi-vitamin_02.jpg',
                    'https://img.friendly-pharmacist.com/YDY/multi_vitamin_03.jpg',
                    'https://img.friendly-pharmacist.com/YDY/multi_vitamin_04.jpg',
                    'https://img.friendly-pharmacist.com/YDY/multi_vitamin_05.jpg','없음','없음','없음','없음'
                   ]
glu_details = ['https://img.friendly-pharmacist.com/YDY/pure_glutathione/pure_glutathione_01.jpg',
               'https://img.friendly-pharmacist.com/YDY/pure_glutathione/pure_glutathione_02.jpg',
               'https://img.friendly-pharmacist.com/YDY/pure_glutathione/pure_glutathione_03.jpg',
               'https://img.friendly-pharmacist.com/YDY/pure_glutathione/pure_glutathione_04.jpg',
               'https://img.friendly-pharmacist.com/YDY/pure_glutathione/pure_glutathione_05.jpg',
               'https://img.friendly-pharmacist.com/YDY/pure_glutathione/pure_glutathione_06.jpg','없음','없음','없음'
              ]
hemchul_details = ['https://img.friendly-pharmacist.com/YDY/active_heme_iron/active_heme_iron_01.jpg',
                   'https://img.friendly-pharmacist.com/YDY/active_heme_iron/active_heme_iron_02.jpg',
                   'https://img.friendly-pharmacist.com/YDY/active_heme_iron/active_heme_iron_03.jpg','없음','없음','없음','없음','없음','없음'
                  ]
postgx_details = ['https://img.friendly-pharmacist.com/YDY/postbiotics_gx/postbiotics_gx_01.jpg',
                 'https://img.friendly-pharmacist.com/YDY/postbiotics_gx/postbiotics_gx_02.jpg',
                 'https://img.friendly-pharmacist.com/YDY/postbiotics_gx/postbiotics_gx_03.jpg','없음','없음','없음','없음','없음','없음'
                ]
rx_macju_details = ['https://img.friendly-pharmacist.com/PhytoRX/leiber_yeast/leiber_yeast_01.jpg',
                    'https://img.friendly-pharmacist.com/PhytoRX/leiber_yeast/leiber_yeast_02.jpg',
                    'https://img.friendly-pharmacist.com/PhytoRX/leiber_yeast/leiber_yeast_03.jpg',
                    'https://img.friendly-pharmacist.com/PhytoRX/leiber_yeast/leiber_yeast_04.jpg',
                    'https://img.friendly-pharmacist.com/PhytoRX/leiber_yeast/leiber_yeast_05.jpg',
                    'https://img.friendly-pharmacist.com/PhytoRX/leiber_yeast/leiber_yeast_06.jpg',
                    'https://img.friendly-pharmacist.com/PhytoRX/leiber_yeast/leiber_yeast_07.jpg',
                    'https://img.friendly-pharmacist.com/PhytoRX/leiber_yeast/leiber_yeast_08.jpg',
                    'https://img.friendly-pharmacist.com/PhytoRX/leiber_yeast/leiber_yeast_09.jpg'
                   ]
rx_billberry_details = ['https://img.friendly-pharmacist.com/PhytoRX/immun_bilberry_renewal/bilberry_new_joint_01.jpg',
                        'https://img.friendly-pharmacist.com/PhytoRX/immun_bilberry_renewal/bilberry_new_joint_02.jpg',
                        'https://img.friendly-pharmacist.com/PhytoRX/immun_bilberry_renewal/bilberry_new_joint_03.jpg',
                        'https://img.friendly-pharmacist.com/PhytoRX/immun_bilberry_renewal/bilberry_new_joint_04.jpg',
                        'https://img.friendly-pharmacist.com/PhytoRX/immun_bilberry_renewal/bilberry_new_joint_05.jpg',
                        'https://img.friendly-pharmacist.com/PhytoRX/immun_bilberry_renewal/bilberry_new_joint_06.jpg',
                        'https://img.friendly-pharmacist.com/PhytoRX/immun_bilberry_renewal/bilberry_new_joint_07.jpg',
                        'https://img.friendly-pharmacist.com/PhytoRX/immun_bilberry_renewal/bilberry_new_joint_08.jpg','없음'
                       ]
sun_details = ['https://img.friendly-pharmacist.com/YDY/renewal_sunvitamin_D/sun_vitamin_01.jpg',
               'https://img.friendly-pharmacist.com/YDY/renewal_sunvitamin_D/sun_vitamin_02.jpg',
               'https://img.friendly-pharmacist.com/YDY/renewal_sunvitamin_D/sun_vitamin_03.jpg',
               'https://img.friendly-pharmacist.com/YDY/renewal_sunvitamin_D/sun_vitamin_04.jpg',
               'https://img.friendly-pharmacist.com/YDY/renewal_sunvitamin_D/sun_vitamin_05.jpg',
               'https://img.friendly-pharmacist.com/YDY/renewal_sunvitamin_D/sun_vitamin_06.jpg',
               'https://img.friendly-pharmacist.com/YDY/renewal_sunvitamin_D/sun_vitamin_07.jpg','없음','없음'
              ]
pure_cumin_details = ['https://img.friendly-pharmacist.com/YDY/pure_red_cumin/pure_red_cumin_01.jpg',
                      'https://img.friendly-pharmacist.com/YDY/pure_red_cumin/pure_red_cumin_02.jpg',
                      'https://img.friendly-pharmacist.com/YDY/pure_red_cumin/pure_red_cumin_03.jpg',
                      'https://img.friendly-pharmacist.com/YDY/pure_red_cumin/pure_red_cumin_04.jpg',
                      'https://img.friendly-pharmacist.com/YDY/pure_red_cumin/pure_red_cumin_05.jpg',
                      'https://img.friendly-pharmacist.com/YDY/pure_red_cumin/pure_red_cumin_06.jpg',
                      'https://img.friendly-pharmacist.com/YDY/pure_red_cumin/pure_red_cumin_07.jpg','없음','없음'
                     ]

In [126]:
result_df[['상세이미지1', '상세이미지2', '상세이미지3', '상세이미지4', '상세이미지5', '상세이미지6', '상세이미지7', '상세이미지8', '상세이미지9']] = np.nan

In [134]:
result_df.loc[result_df['상품명'].str.contains('마그듀오'), ['상세이미지1', '상세이미지2', '상세이미지3', '상세이미지4', '상세이미지5', '상세이미지6', '상세이미지7', '상세이미지8', '상세이미지9']] = magduo_details
result_df.loc[result_df['상품명'].str.contains('내몸에'), ['상세이미지1', '상세이미지2', '상세이미지3', '상세이미지4', '상세이미지5', '상세이미지6', '상세이미지7', '상세이미지8', '상세이미지9']] = hyoso_details
result_df.loc[result_df['상품명'].str.contains('썬'), ['상세이미지1', '상세이미지2', '상세이미지3', '상세이미지4', '상세이미지5', '상세이미지6', '상세이미지7', '상세이미지8', '상세이미지9']] = sun_details
result_df.loc[result_df['상품명'].str.contains('큐텐'), ['상세이미지1', '상세이미지2', '상세이미지3', '상세이미지4', '상세이미지5', '상세이미지6', '상세이미지7', '상세이미지8', '상세이미지9']] = bq10_details
result_df.loc[result_df['상품명'].str.contains('헴철'), ['상세이미지1', '상세이미지2', '상세이미지3', '상세이미지4', '상세이미지5', '상세이미지6', '상세이미지7', '상세이미지8', '상세이미지9']] = hemchul_details
result_df.loc[result_df['상품명'].str.contains('오큐클리어'), ['상세이미지1', '상세이미지2', '상세이미지3', '상세이미지4', '상세이미지5', '상세이미지6', '상세이미지7', '상세이미지8', '상세이미지9']] = ocuclear_details
result_df.loc[result_df['상품명'].str.contains('코랄칼마디'), ['상세이미지1', '상세이미지2', '상세이미지3', '상세이미지4', '상세이미지5', '상세이미지6', '상세이미지7', '상세이미지8', '상세이미지9']] = coral_details
result_df.loc[result_df['상품명'].str.contains('파이토 레드큐민'), ['상세이미지1', '상세이미지2', '상세이미지3', '상세이미지4', '상세이미지5', '상세이미지6', '상세이미지7', '상세이미지8', '상세이미지9']] = red_cumin_details
result_df.loc[result_df['상품명'].str.contains('포스트바이오틱스'), ['상세이미지1', '상세이미지2', '상세이미지3', '상세이미지4', '상세이미지5', '상세이미지6', '상세이미지7', '상세이미지8', '상세이미지9']] = postgx_details
result_df.loc[result_df['상품명'].str.contains('폴라'), ['상세이미지1', '상세이미지2', '상세이미지3', '상세이미지4', '상세이미지5', '상세이미지6', '상세이미지7', '상세이미지8', '상세이미지9']] = omega3_details
result_df.loc[result_df['상품명'].str.contains('퓨어 레드큐민'), ['상세이미지1', '상세이미지2', '상세이미지3', '상세이미지4', '상세이미지5', '상세이미지6', '상세이미지7', '상세이미지8', '상세이미지9']] = pure_cumin_details
result_df.loc[result_df['상품명'].str.contains('글루타치온'), ['상세이미지1', '상세이미지2', '상세이미지3', '상세이미지4', '상세이미지5', '상세이미지6', '상세이미지7', '상세이미지8', '상세이미지9']] = glu_details
result_df.loc[result_df['상품명'].str.contains('비타민C'), ['상세이미지1', '상세이미지2', '상세이미지3', '상세이미지4', '상세이미지5', '상세이미지6', '상세이미지7', '상세이미지8', '상세이미지9']] = vitaminc_details
result_df.loc[result_df['상품명'].str.contains('프로바이오'), ['상세이미지1', '상세이미지2', '상세이미지3', '상세이미지4', '상세이미지5', '상세이미지6', '상세이미지7', '상세이미지8', '상세이미지9']] = slb_details
result_df.loc[result_df['상품명'].str.contains('라이버'), ['상세이미지1', '상세이미지2', '상세이미지3', '상세이미지4', '상세이미지5', '상세이미지6', '상세이미지7', '상세이미지8', '상세이미지9']] = rx_macju_details
result_df.loc[result_df['상품명'].str.contains('이뮨'), ['상세이미지1', '상세이미지2', '상세이미지3', '상세이미지4', '상세이미지5', '상세이미지6', '상세이미지7', '상세이미지8', '상세이미지9']] = rx_billberry_details
result_df.loc[result_df['상품명'].str.contains('멀티비타민'), ['상세이미지1', '상세이미지2', '상세이미지3', '상세이미지4', '상세이미지5', '상세이미지6', '상세이미지7', '상세이미지8', '상세이미지9']] = fullspec_details

/var/folders/wl/txjbcgld2sngchsc7txxqppm0000gp/T/ipykernel_83562/2410871335.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'https://img.friendly-pharmacist.com/YDY/magduo/magduo_01.jpg' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  result_df.loc[result_df['상품명'].str.contains('마그듀오'), ['상세이미지1', '상세이미지2', '상세이미지3', '상세이미지4', '상세이미지5', '상세이미지6', '상세이미지7', '상세이미지8', '상세이미지9']] = magduo_details
/var/folders/wl/txjbcgld2sngchsc7txxqppm0000gp/T/ipykernel_83562/2410871335.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'https://img.friendly-pharmacist.com/YDY/magduo/magduo_02.jpg' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  result_df.loc[result_df['상품명'].str.contains('마그듀오'), ['상세이미지1', '상세이미지2', '상세이미지3', '상세이미지4', '상세이미지5', '상세이미지

In [140]:
result_df['썸네일'] = '있음'
result_df['상세 이미지'] = '있음'
result_df['고시 이미지'] = '없음'

In [146]:
result_df.to_excel('/Users/seanyu_mac/Downloads/친한스토어_상품생성_스크립트.xlsx')

In [173]:
result_df.index.size - 37

593

In [249]:
con1 = pb_df['brand_id'] == 684
con2 = pb_df['brand_id'] == 2631
pb_df = pb_df[con1 | con2]

In [251]:
!pip install thefuzz

from thefuzz import process
matches = []
for result_name in result_df['상품명']:
    best_match, score, _ = process.extractOne(result_name, pb_df['product_name'])
    matches.append((result_name, best_match, score))
                                        